## Prepare get_stats and merge function

In [86]:
def get_stats(ids, counts=None):
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    
    return counts

In [20]:
text = "Hello World!"
tokens = text.encode("utf-8")
print(tokens)
tokens = list(tokens)
print(tokens)

b'Hello World!'
[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 33]


In [5]:
stats = get_stats(tokens)
print(stats)

{(72, 101): 1, (101, 108): 1, (108, 108): 1, (108, 111): 1, (111, 32): 1, (32, 87): 1, (87, 111): 1, (111, 114): 1, (114, 108): 1, (108, 100): 1, (100, 33): 1}


In [102]:
def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

In [7]:
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 101))

[5, 6, 101, 9, 1]


## Step 1: BasicTokenizer class

Write the BasicTokenizer class, with the following three core functions:

- def train(self, text, vocab_size, verbose=False)
- def encode(self, text)
- def decode(self, ids)

In [19]:
vocab = {idx: bytes([idx]) for idx in range(256)}

print(vocab[98])

tokens = b"".join(vocab[idx] for idx in range(110))
print(tokens)


b'b'
b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklm'


In [38]:
class BasicTokenizer:
    def __init__(self):
        super().__init__()

    def train(self, text, vocab_size, verbose=False):
        
        raw_bytes = text.encode("utf-8")  # converting text to raw bytes using utf-8 encoding
        ids = list(raw_bytes)  # converting to list of integers in range 0..255

        assert vocab_size >= 256
        num_merges = vocab_size - 256

        merges = {} # (int, int) : int
        vocab = {idx: bytes([idx]) for idx in range(256)}  # int : bytes
        
        for i in range(num_merges):
            # count the number of frequency of consecutive pairs
            stats = get_stats(ids)
            # chose the pair with the highest count 
            pair = max(stats, key=stats.get)
            # create new token : assignt it next available id
            idx = i + 256
            
            # replace all occurances of pair in ids with idx
            ids = merge(ids, pair, idx)

            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]

            # prints 
            if verbose:
                print(f'merge {i+1}/{num_merges} : {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurences')
        
        # save merges and vocab
        self.merges = merges  # used in encode()
        self.vocab = vocab  # used in decode()

    def encode(self, text):
        ''' given a string, return list of integers (the tokens)'''
        
        ids = list(text.encode("utf-8")) 
        while len(ids) >= 2:
            stats = get_stats(ids)
            # find pair with lowest idx in merges
            pair = min(stats, key=lambda p : self.merges.get(p, float("inf")))
            
            if pair not in self.merges:
                break # nothing else can be merged

            # identify the index from merges
            idx = self.merges[pair]
            # merge the pair(lowest merge index)
            ids = merge(ids, pair, idx)
        return ids

    def decode(self, ids):
        ''' given list of integers(token ids), return python string '''
        # iterating through ids and finding appropriate bytes for that idx from vocab
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        # just decode using utf-8
        text = text_bytes.decode("utf-8", errors="replace")
        return text

### Train your tokenizer 

One default test you may wish to use is the text file tests/taylorswift.txt.



In [39]:
tokenizer = BasicTokenizer()

In [40]:
import os

with open("taylorswift.txt", 'r') as f:
    raw_text = f.read()

print("the length of text: ", len(raw_text))
#print(raw_text[850 : 2050])

the length of text:  185768


In [41]:
tokenizer.train(raw_text, vocab_size=500, verbose=True)

merge 1/244 : (101, 32) -> 256 (b'e ') had 2981 occurences
merge 2/244 : (44, 32) -> 257 (b', ') had 2961 occurences
merge 3/244 : (100, 32) -> 258 (b'd ') had 2617 occurences
merge 4/244 : (46, 32) -> 259 (b'. ') had 2560 occurences
merge 5/244 : (114, 32) -> 260 (b'r ') had 2428 occurences
merge 6/244 : (50, 48) -> 261 (b'20') had 2365 occurences
merge 7/244 : (115, 32) -> 262 (b's ') had 2053 occurences
merge 8/244 : (105, 110) -> 263 (b'in') had 2006 occurences
merge 9/244 : (111, 110) -> 264 (b'on') had 1815 occurences
merge 10/244 : (114, 105) -> 265 (b'ri') had 1805 occurences
merge 11/244 : (116, 32) -> 266 (b't ') had 1802 occurences
merge 12/244 : (116, 104) -> 267 (b'th') had 1737 occurences
merge 13/244 : (101, 258) -> 268 (b'ed ') had 1736 occurences
merge 14/244 : (257, 261) -> 269 (b', 20') had 1705 occurences
merge 15/244 : (97, 110) -> 270 (b'an') had 1487 occurences
merge 16/244 : (97, 114) -> 271 (b'ar') had 1360 occurences
merge 17/244 : (101, 260) -> 272 (b'er ') h

In [64]:
test = "Taylor Swift won Grammy music award"
test_ids = tokenizer.encode(test)
print(test_ids)

[345, 119, 279, 71, 114, 362, 109, 273, 109, 433, 97, 119, 355]


In [68]:
tokenizer.decode(tokenizer.encode(test))

'Taylor Swift won Grammy music award'

In [69]:
tokenizer.decode(tokenizer.encode(test)) == "Taylor Swift won Grammy music award"

True

## Step 2 (BasicTokenizer -> RegexTokenizer)

Convert you BasicTokenizer into a RegexTokenizer, which takes a regex pattern and splits the text exactly as GPT-4 would. Process the parts separately as before, then concatenate the results. Retrain your tokenizer and compare the results before and after. You should see that you will now have no tokens that go across categories (numbers, letters, punctuation, more than one whitespace). Use the GPT-4 pattern:

GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [70]:
import regex as re

In [73]:
# the main GPT text split patterns, see
# https://github.com/openai/tiktoken/blob/main/tiktoken_ext/openai_public.py
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [75]:
gpt2pat = re.compile(GPT2_SPLIT_PATTERN)

print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))
text_chunks = re.findall(gpt2pat, "Hello've world123 how's are you!!!?")
print(text_chunks)  # we have here text chunks splitted by GPT2 split pattern

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']
['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


In [87]:
# creating ids for each text chunks seperately, ids will contain list of integers for each text chunks
ids = [list(ch.encode("utf-8")) for ch in text_chunks]
print(ids)

[[72, 101, 108, 108, 111], [39, 118, 101], [32, 119, 111, 114, 108, 100], [49, 50, 51], [32, 104, 111, 119], [39, 115], [32, 97, 114, 101], [32, 121, 111, 117], [33, 33, 33, 63]]


In [89]:
stats = {}

for chunk_ids in ids:
    get_stats(chunk_ids, stats)
    
print(stats)

{(72, 101): 1, (101, 108): 1, (108, 108): 1, (108, 111): 1, (39, 118): 1, (118, 101): 1, (32, 119): 1, (119, 111): 1, (111, 114): 1, (114, 108): 1, (108, 100): 1, (49, 50): 1, (50, 51): 1, (32, 104): 1, (104, 111): 1, (111, 119): 1, (39, 115): 1, (32, 97): 1, (97, 114): 1, (114, 101): 1, (32, 121): 1, (121, 111): 1, (111, 117): 1, (33, 33): 2, (33, 63): 1}


### Let's write RegexTokenizer class

In [114]:
GPT2_SPLIT_PATTERN = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

class RegexTokenizer:
    def __init__(self, pattern = None):
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN
        self.compiled_pattern = re.compile(self.pattern)

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # split text to text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # taking ids
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx:bytes([idx]) for idx in range(256)}

        stats = {}
        for i in range(num_merges):
            # count the frequency occurance of pairs
            for chunk_ids in ids:
                stats = get_stats(chunk_ids, stats)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            idx = 256 + i
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]

            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")
        
        # save the variables
        self.merges = merges # using for encode()
        self.vocab = vocab
    
    def _encode_chunk(self, text_bytes):
        ''' return the token ids'''
        
        ids = list(text_bytes) 
        while len(ids) >= 2:
            stats = get_stats(ids)
            # find pair with lowest idx in merges
            pair = min(stats, key=lambda p : self.merges.get(p, float("inf")))
            
            if pair not in self.merges:
                break # nothing else can be merged

            # identify the index from merges
            idx = self.merges[pair]
            # merge the pair(lowest merge index)
            ids = merge(ids, pair, idx)
        return ids

    def encode(self, text):
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)

        # all chunks of text are encoded seperately, then results will be joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8")  # raw bytes
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids
    
    def decode(self, ids):
        # given ids (list of integers), return Python string
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

In [115]:
regex_tokenizer = RegexTokenizer()

In [116]:
import os

with open("taylorswift.txt", 'r') as f:
    raw_text = f.read()

print("the length of text: ", len(raw_text))
#print(raw_text[850 : 2050])

the length of text:  185768


In [117]:
regex_tokenizer.train(raw_text, vocab_size=700, verbose=True)

merge 1/444: (101, 114) -> 256 (b'er') had 2359 occurrences
merge 2/444: (50, 48) -> 257 (b'20') had 4374 occurrences
merge 3/444: (111, 114) -> 258 (b'or') had 6228 occurrences
merge 4/444: (105, 110) -> 259 (b'in') had 8024 occurrences
merge 5/444: (101, 100) -> 260 (b'ed') had 9380 occurrences
merge 6/444: (104, 101) -> 261 (b'he') had 11038 occurrences
merge 7/444: (32, 116) -> 262 (b' t') had 12768 occurrences
merge 8/444: (111, 110) -> 263 (b'on') had 14520 occurrences
merge 9/444: (32, 83) -> 264 (b' S') had 14697 occurrences
merge 10/444: (97, 114) -> 265 (b'ar') had 15190 occurrences
merge 11/444: (97, 110) -> 266 (b'an') had 16357 occurrences
merge 12/444: (32, 97) -> 267 (b' a') had 17183 occurrences
merge 13/444: (114, 105) -> 268 (b'ri') had 17408 occurrences
merge 14/444: (32, 65) -> 269 (b' A') had 18690 occurrences
merge 15/444: (32, 65) -> 270 (b' A') had 18690 occurrences
merge 16/444: (32, 65) -> 271 (b' A') had 18690 occurrences
merge 17/444: (97, 108) -> 272 (b'al'

In [118]:
txt = "Taylor Swift is a famous star in the world"
print(regex_tokenizer.encode(txt))

[84, 376, 320, 279, 32, 390, 267, 277, 466, 321, 115, 32, 273, 265, 363, 284, 355, 258, 108, 100]


In [119]:
txt = "Taylor Swift is a famous star in the world"
ids = regex_tokenizer.encode(txt)
print(ids)

[84, 376, 320, 279, 32, 390, 267, 277, 466, 321, 115, 32, 273, 265, 363, 284, 355, 258, 108, 100]


In [133]:
regex_tokenizer.decode(ids)

'Taylor Swift is a famous star in the world'

In [134]:
txt == regex_tokenizer.decode(regex_tokenizer.encode(txt))

True

### Compare BasicTokenizer with RegexTokenizer 

In [135]:
txt = "Taylor Swift is a famous star in the world"
ids_reg = regex_tokenizer.encode(txt)
ids_basic = tokenizer.encode(txt)

print(ids_reg)
print(ids_basic)

[84, 376, 320, 279, 32, 390, 267, 277, 466, 321, 115, 32, 273, 265, 363, 284, 355, 258, 108, 100]
[345, 105, 262, 338, 102, 362, 321, 262, 339, 97, 260, 263, 288, 119, 291, 108, 100]


In [136]:
txt == regex_tokenizer.decode(regex_tokenizer.encode(txt))
txt == tokenizer.decode(tokenizer.encode(txt))

True

## Step 3 

You're now ready to load the merges from the GPT-4 tokenizer and show that your tokenizer produces the identical results for both encode and decode, matching tiktoken.



In [142]:
# match this
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back
print(text)


hello world!!!? (안녕하세요!) lol123 😉


In [143]:
text == "hello world!!!? (안녕하세요!) lol123 😉"

True

In [146]:
# tokenizer_reg = RegexTokenizer()
# ids_my = tokenizer_reg.encode("hello world!!!? (안녕하세요!) lol123 😉")  # not encoded

**Unfortunately, you will run into two issues:**

- It is not trivial to recover the raw merges from the GPT-4 tokenizer. You can easily recover what we call vocab here, and what they call and store under enc._mergeable_ranks. Feel free to copy paste the recover_merges function in minbpe/gpt4.py, which takes these ranks and returns the raw merges. If you wish to know how this function works, read this and this. Basically, under some conditions it is enough to only store the parent nodes (and their rank) and get rid of the precise details of which children merged up to any parent.

- Second, the GPT-4 tokenizer for some reason permutes its raw bytes. It stores this permutation in the first 256 elements of the mergeable ranks, so you can recover this byte shuffle relatively simply as byte_shuffle = {i: enc._mergeable_ranks[bytes([i])] for i in range(256)}. In both your encode and decode, you'll have to shuffle bytes around accordingly. If you're stuck, reference the minbpe/gpt4.py` file for hints.